In [1]:
import pandas as pd
import os
import sys
from dotenv import load_dotenv

In [19]:
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
scripts_path = os.path.join(project_root, 'Scripts')

sys.path.append(scripts_path)

from db_tools import run_query, fill_db, create_table_query

In [20]:
load_dotenv()

# Access the values using os.environ.get()
connection_details = {
    "user": os.environ.get("DB_USER"),
    "password": os.environ.get("DB_PASSWORD"),
    "database": os.environ.get("DB_NAME"),
    "host": os.environ.get("DB_HOST"),
    "port": os.environ.get("DB_PORT")
}

In [22]:
# Define the root directory
root_directory = './../Data/'


sql_queries = ""


totals_table_created = False  # Flag to track if Totals table has been created
all_totals_data = pd.DataFrame()  # DataFrame to combine all Totals data

# Iterate through each folder in the root directory
for folder_name in os.listdir(root_directory):
    folder_path = os.path.join(root_directory, folder_name)
    print(folder_name)

    # Check if the item is a directory
    if os.path.isdir(folder_path):
        chart_data_path = os.path.join(folder_path, 'Chart data.csv')
        table_data_path = os.path.join(folder_path, 'Table data.csv')
        totals_data_path = os.path.join(folder_path, 'Totals.csv')


        

        # # Check if "Chart data.csv" or "Table data.csv" exists in the folder
        if os.path.exists(chart_data_path):
            chart_data_df = pd.read_csv(chart_data_path)
            table_name = f'{folder_name.lower().replace(" ","_")}_chart_data'
            table_query = create_table_query(chart_data_df, table_name)
            sql_queries += table_query
            print(table_query)
            # Process chart data
   
            run_query(connection_details, table_query)  # Create table if not exists
            fill_db(connection_details, chart_data_df, table_name)

        if os.path.exists(table_data_path):
            # Process table data
            table_data_df = pd.read_csv(table_data_path)
            table_name = f'{folder_name.lower().replace(" ","_")}_table_data'

            table_query = create_table_query(table_data_df, table_name)
            sql_queries += table_query
            print(table_query)


            run_query(connection_details, table_query)  # Create table if not exists
            fill_db(connection_details, table_data_df, table_name)
        
        # Check if "Totals.csv" exists in the folder
        if os.path.exists(totals_data_path):
            totals_df = pd.read_csv(totals_data_path)
            all_totals_data = pd.concat([all_totals_data, totals_df], ignore_index=True)
        
    


# Create the CREATE TABLE query for totals_table_data
totals_table_query = create_table_query(all_totals_data, 'totals_table_data')
sql_queries += totals_table_query
print(totals_table_query)
# Create the totals_table_data table

# Specify the file path for the SQL file
sql_file_path = './sql_queries.sql'

# Write the SQL queries to the file
with open(sql_file_path, 'w') as sql_file:
    sql_file.write(sql_queries)

print(f'SQL queries saved to {sql_file_path}')

run_query(connection_details, totals_table_query)

# Insert the combined Totals data into totals_table_data
fill_db(connection_details, all_totals_data, 'totals_table_data')

Subscription status
CREATE TABLE IF NOT EXISTS "subscription_status_chart_data" ("Date" TEXT, "Subscription status" TEXT, "Views" INTEGER, PRIMARY KEY ("Date", "Subscription status"));
'dbname'
CREATE TABLE IF NOT EXISTS "subscription_status_table_data" ("Subscription status" TEXT, "Views" INTEGER, "Watch time (hours)" REAL, "Average view duration" TEXT, PRIMARY KEY ("Subscription status"));
'dbname'
Sharing service
CREATE TABLE IF NOT EXISTS "sharing_service_chart_data" ("Date" TEXT, "Sharing service" TEXT, "Shares" INTEGER, PRIMARY KEY ("Date", "Sharing service"));
'dbname'
CREATE TABLE IF NOT EXISTS "sharing_service_table_data" ("Sharing service" TEXT, "Shares" INTEGER, PRIMARY KEY ("Sharing service"));
'dbname'
New and returning viewers
CREATE TABLE IF NOT EXISTS "new_and_returning_viewers_chart_data" ("Date" TEXT, "New and returning viewers" TEXT, "Views" INTEGER, PRIMARY KEY ("Date", "New and returning viewers"));
'dbname'
CREATE TABLE IF NOT EXISTS "new_and_returning_viewers_tab

1